<a href="https://colab.research.google.com/github/alihuss1017/LSTM-Weather-Prediction/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import userdata
gh_token = userdata.get('GITHUB_TOKEN')

In [12]:
!git clone https://{gh_token}@github.com/alihuss1017/LSTM-Weather-Prediction.git

Cloning into 'LSTM-Weather-Prediction'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 28 (delta 11), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 92.36 KiB | 8.40 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [13]:
cd LSTM-Weather-Prediction

/content/LSTM-Weather-Prediction


In [14]:
import pandas as pd
df = pd.read_csv('data/seattle-weather.csv')

## Checking for Null Values and Duplicates

In [15]:
print(f'''Number of null values:\n{df.isnull().sum()}\n\nNumber of duplicated rows: {df.duplicated().sum()}''')

Number of null values:
date             0
precipitation    0
temp_max         0
temp_min         0
wind             0
weather          0
dtype: int64

Number of duplicated rows: 0


## Setting DateTime as Index

In [16]:
df = df.set_index(df["date"])
df = df.drop('date', axis = 1)


## Saving the mean and standard deviation for Inference Purposes

In [17]:
mu, std = df['temp_max'].mean(), df['temp_max'].std()

## One-Hot Encoding Categorical Features:

In [18]:
df_encoded = pd.get_dummies(df, columns = ['weather'])

## Applying Z-Score Normalization on Numerical Features

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
columns_to_normalize = df_encoded.select_dtypes(include='float').columns.tolist()

df_encoded[columns_to_normalize] = scaler.fit_transform(df_encoded[columns_to_normalize])

## Defining the PyTorch Custom Dataset Class

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader

class WeatherDataset(Dataset):

  def __init__(self, data_df, seq_len):
    self.data = data_df
    self.seq_len = seq_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    cols_to_cast = self.data.select_dtypes(include = ['object', 'bool']).columns.tolist()
    for col in cols_to_cast:
      self.data[col] = self.data[col].astype('int')

    x = torch.tensor(self.data.iloc[idx:idx+self.seq_len].values, dtype = torch.float32)
    y = torch.tensor(self.data['temp_max'].iloc[idx+self.seq_len+1], dtype = torch.float32)

    return x, y


## Defining Model

In [21]:
import torch.nn as nn

input_features = 9

class lstmModel(nn.Module):
  def __init__(self, hidden_features, num_layers):
    super().__init__()

    self.lstm = nn.LSTM(input_size = input_features,
                        hidden_size = hidden_features, num_layers = num_layers,
                        batch_first = True)
    self.fc = nn.Linear(hidden_features, 1)

  def forward(self, x):
    out, [h, c] = self.lstm(x)
    return self.fc(h[-1])


## Model Debugging

In [22]:
model = lstmModel(32, 2)
model.eval()
with torch.no_grad():
  print(f'Output: {model(torch.rand((5, 9)))}')

Output: tensor([0.1446])


## Configuring Device

In [23]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

print(f'Device: {device}')

Device: cuda


## Training Model

In [24]:
def train(model, train_loader, optimizer, loss_fn):
  model.train()
  total_loss = 0
  for X, y in train_loader:
    optimizer.zero_grad()

    X, y = X.to(device), y.to(device)
    y_hat = model(X)

    loss = loss_fn(y, y_hat)
    total_loss += loss.item()

    loss.backward()
    optimizer.step()

  return total_loss / len(train_loader)


## Evaluating Model

In [25]:
import matplotlib.pyplot as plt
import numpy as np


def eval(model, val_loader, loss_fn):
  predicted = []
  actual = []
  total_loss = 0
  model.eval()

  with torch.no_grad():
    for X, y in val_loader:
      X, y = X.to(device), y.to(device)
      y_hat = model(X)

      loss = loss_fn(y, y_hat)
      total_loss += loss.item()

  return total_loss / len(val_loader)




In [26]:
!pip install wandb
!pip install optuna

In [27]:
!wandb login

wandb: Currently logged in as: alihuss1017 (alihuss1017-uc-san-diego) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [28]:
!pip install optuna-integration[wandb]

In [29]:
from optuna.integration.wandb import WeightsAndBiasesCallback

In [30]:
import optuna
import wandb
from torch.utils.data import Subset
import torch.optim as optim


def objective(trial):

  seq_len = trial.suggest_int('seq_len', 5, 20)
  batch_size = trial.suggest_int('batch_size', 16, 64)
  hidden_features = trial.suggest_int('hidden_features', 32, 128)
  num_layers = trial.suggest_int('num_layers', 1, 3)
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
  num_epochs = trial.suggest_int('num_epochs', 5, 10)
  loss_fn = nn.MSELoss()

  wandb.init(
      project="lstm-weather",
      config={
          'seq_len': seq_len,
          'batch_size': batch_size,
          'hidden_features': hidden_features,
          'num_layers': num_layers,
          'lr': lr,
          'num_epochs': num_epochs
      }
  )

  dataset = WeatherDataset(data_df = df_encoded, seq_len = seq_len)
  train_len = int(0.7 * len(dataset))

  train_data = Subset(dataset, range(train_len))
  val_data = Subset(dataset, range(train_len, len(dataset)))

  train_loader = DataLoader(train_data, batch_size = 32, num_workers = 2, drop_last = True)
  val_loader = DataLoader(val_data, batch_size = 32, num_workers = 2, drop_last = True )

  model = lstmModel(hidden_features = hidden_features, num_layers = num_layers).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr = lr)

  for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, loss_fn)
    val_loss = eval(model, val_loader, loss_fn)

    wandb.log({"epoch": epoch, "train_loss": train_loss, "val_loss": val_loss})

  return val_loss



In [ ]:
wandb_callback = WeightsAndBiasesCallback(metric_name="val_loss", wandb_kwargs={'project': 'lstm-weather'})

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, callbacks=[wandb_callback])

best_trial = study.best_trial

/tmp/ipython-input-813855177.py:1: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandb_callback = WeightsAndBiasesCallback(metric_name="val_loss", wandb_kwargs={'project': 'lstm-weather'})
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: alihuss1017 (alihuss1017-uc-san-diego) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[I 2025-11-02 19:41:56,876] A new study created in memory with name: no-name-26a3c286-60d5-4386-b72b-1ec9e6f0d91e
/tmp/ipython-input-2864950579.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[I 2025-11-02 19:42:13,412] Trial 0 finished with value: 0.29073649186354417 and parameters: {'seq_len': 13, 'batch_size': 42, 'hidden_features': 62, 'num_layers': 3, 'lr': 0.007236687794546415, 'num_epochs': 10}. Best is trial 0 with value: 0.29073649186354417.
/tmp/ipython-input-2864950579.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▂▂▂▁▁▁▁▁▁
val_loss,█▄▆▃▂▁▂▂▁▁
epoch,9
train_loss,0.30283
val_loss,0.29074


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[I 2025-11-02 19:42:19,789] Trial 1 finished with value: 0.36227999742214495 and parameters: {'seq_len': 10, 'batch_size': 45, 'hidden_features': 102, 'num_layers': 3, 'lr': 0.00786144404549479, 'num_epochs': 5}. Best is trial 0 with value: 0.29073649186354417.
/tmp/ipython-input-2864950579.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)


...

batch_size,▁
epoch,▁▂▃▃▄▅▆▆▇█
hidden_features,▁
lr,▁
num_epochs,▁
num_layers,▁
seq_len,▁
train_loss,█▃▂▁▁▁▁▁▁▁
val_loss,█▇▃▂▂▂▂▂▁▁▁
batch_size,52
epoch,9


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[I 2025-11-02 19:44:26,116] Trial 15 finished with value: 0.2925184529561263 and parameters: {'seq_len': 18, 'batch_size': 64, 'hidden_features': 75, 'num_layers': 2, 'lr': 0.0008069618237283487, 'num_epochs': 9}. Best is trial 11 with value: 0.2885194667256795.
/tmp/ipython-input-2864950579.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)


batch_size,▁
epoch,▁▂▃▄▅▅▆▇█
hidden_features,▁
lr,▁
num_epochs,▁
num_layers,▁
seq_len,▁
train_loss,█▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁▁▁▁
batch_size,64
epoch,8


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[I 2025-11-02 19:44:36,621] Trial 16 finished with value: 0.29797426668497234 and parameters: {'seq_len': 10, 'batch_size': 38, 'hidden_features': 62, 'num_layers': 2, 'lr': 0.00339200990906664, 'num_epochs': 10}. Best is trial 11 with value: 0.2885194667256795.
/tmp/ipython-input-2864950579.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)


batch_size,▁
epoch,▁▂▃▃▄▅▆▆▇█
hidden_features,▁
lr,▁
num_epochs,▁
num_layers,▁
seq_len,▁
train_loss,█▂▂▂▁▂▁▂▁▁
val_loss,█▆▅▃▂▂▃▂▁▁▁
batch_size,38
epoch,9


In [1]:
for key, val in best_trial.params.items():
  print(f"{key}: {val}")

NameError: name 'best_trial' is not defined